In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
import os
sys.path.insert(0,'../')

In [2]:
from src.mlflow_setup import mlflow
from evaluate import load
import tempfile


/home/jfeil/.pyenv/versions/3.10.12/envs/MasterThesis-NLP/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/jfeil/.pyenv/versions/3.10.12/envs/MasterThesis-NLP/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/jfeil/.pyenv/versions/3.10.12/envs/MasterThesis-NLP/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


# Data Loading

In [3]:
experiment_ids = [0]
runs = []
for experiment_id in experiment_ids:
    runs += mlflow.search_runs(experiment_ids=str(experiment_id))['run_id'].values.tolist()
runs

['cb8135f768dd4a8285257deb0bc3d912',
 '5565790ff7c74ef69997fc0af6a3cd06',
 '41ef3452981a4b208aa35c8f8b7197df',
 '14ae3307f7df4c26b249b847ef24c26a',
 'ebee22c9aef14c8baaca0ec3192e8833',
 '9eaf7a3028bd4b2cb773d0105f165add',
 '7e5d6c8cba4d49a8853e5e4c06072da7']

In [5]:
import json

with tempfile.TemporaryDirectory() as temp_path:
    mlflow.artifacts.download_artifacts(run_id='cb8135f768dd4a8285257deb0bc3d912', dst_path=temp_path)
    with open(os.path.join(temp_path, 'eval_results_table.json')) as file:
        eval_data = json.load(file)
    

# Scores

In [9]:
index = 412
index = 42

predictions = eval_data['data'][index][3].replace('</s>', '')
targets = eval_data['data'][index][2]
print(eval_data['data'][index])

print("X: " + str(predictions))
print("Y: " + str(targets))

["Chaos --- „Es war da ein Gefühl oder, richtiger gesagt, ein ganzes ''Chaos'' von Gefühlen, in dem ich mich naturgemäß verirren mußte.“", {'title': 'Chaos', 'context_word': 'Chaos', 'context_sentence': '„Es war da ein Gefühl oder, richtiger gesagt, ein ganzes Chaos von Gefühlen, in dem ich mich naturgemäß verirren mußte.“'}, 'Zustand der Unordnung', 'Zustand großer Unordnung und Verwirrung.</s>', 17, 0.0012143633, 10.0, 18.8, 0.4]
X: Zustand großer Unordnung und Verwirrung.
Y: Zustand der Unordnung


## MoverScore

In [71]:
# pip install git+https://github.com/jfeil/emnlp19-moverscore.git --force

In [7]:
os.environ['MOVERSCORE_MODEL'] = "distilbert-base-german-cased"

os.environ['MOVERSCORE_MODEL'] = "distilbert-base-multilingual-cased"
# os.environ['MOVERSCORE_MODEL'] = "albert-base-v2"


from moverscore_v2 import get_idf_dict, word_mover_score 
from collections import defaultdict

def calc_sentence_score(prediction, target):
    idf_dict_hyp = defaultdict(lambda: 1.)
    idf_dict_ref = defaultdict(lambda: 1.)
    
    
    return word_mover_score([target], [prediction], idf_dict_ref, idf_dict_hyp, \
                              stop_words=[], n_gram=1, remove_subwords=True)

In [8]:
from tqdm.notebook import tqdm

scores = 0
max_score = -1
max_index = -1
min_score = 2
min_index = -1

for i in tqdm(range(len(eval_data['data']))):
    new_score = calc_sentence_score(eval_data['data'][i][3].replace('</s>', ''), eval_data['data'][i][2])[0]

    if new_score > max_score:
        max_score = new_score
        max_index = i
    if new_score < min_score:
        min_score = new_score
        min_index = i
        
    scores += new_score

max_score, max_index, min_score, min_index

  0%|          | 0/1107 [00:00<?, ?it/s]

(0.6345617059763319, 42, 0.43803306599011094, 897)

In [52]:
# cb8135f768dd4a8285257deb0bc3d912 --> 0.5240697396879908
# 5565790ff7c74ef69997fc0af6a3cd06 --> 0.5254168598952033
# 41ef3452981a4b208aa35c8f8b7197df --> 0.5104922168193816
# 14ae3307f7df4c26b249b847ef24c26a --> 0.5018933512907802
# ebee22c9aef14c8baaca0ec3192e8833 --> 0.5103564892456643
# 9eaf7a3028bd4b2cb773d0105f165add --> 0.5231842858875173
# 7e5d6c8cba4d49a8853e5e4c06072da7 --> 0.5239859657285223

scores / len(eval_data['data'])

0.5239859657285223

## BERTscore

In [31]:
bertscore = load("bertscore")

In [51]:
scores = bertscore.compute(predictions=predictions, references=targets, lang="de")
scores

{'precision': [0.7915669083595276],
 'recall': [0.7916980385780334],
 'f1': [0.7916324734687805],
 'hashcode': 'bert-base-multilingual-cased_L9_no-idf_version=0.3.12(hug_trans=4.36.1)'}

In [9]:
## ROUGE-L1

In [10]:
## Token Analysis